In [0]:
import torch
from torchtext import data

In [0]:
TEXT = data.Field(include_lengths = True) #tokenize -> Default: string.split
LABEL = data.LabelField(dtype = torch.float)

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
train_data = data.TabularDataset('/content/gdrive/My Drive/Colab Notebooks/pytorch/nsmc/ratings_train.txt', 
                                'tsv', [('id', None), ('text', TEXT), ('label', LABEL)],
                                 skip_header = True, filter_pred = lambda x: True if len(x.text) > 1 else False)
test_data = data.TabularDataset('/content/gdrive/My Drive/Colab Notebooks/pytorch/nsmc/ratings_test.txt',
                                'tsv', [('id', None), ('text', TEXT), ('label', LABEL)],
                                skip_header = True, filter_pred = lambda x: True if len(x.text) > 1 else False)

In [5]:
print(vars(train_data[0]))
print(vars(test_data[0]))

{'text': ['아', '더빙..', '진짜', '짜증나네요', '목소리'], 'label': '0'}
{'text': ['굳', 'ㅋ'], 'label': '1'}


In [0]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(0))

In [7]:
print(len(train_data))
print(len(valid_data))

96333
41286


In [0]:
MAX_VOCAB_SIZE = 50000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE,
                 vectors = "glove.6B.200d",
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size = BATCH_SIZE, device = device, sort_within_batch = True, sort_key = lambda x: len(x.text))

In [0]:
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class LSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim,
                 num_layers, bidirectional, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=pad_idx)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers,
                           bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        return self.fc(hidden)
    

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 200
HIDDEN_DIM = 256
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
NUM_LAYERS = 2
BIDIRECTIONAL = True

In [0]:
model = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, NUM_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)
model = model.to(device)

In [0]:
def count_parameters(model):
     return sum(p.numel() for p in model.parameters() if p.requires_grad) 

In [0]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [0]:
import torch.optim as optim

In [0]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
criterion = criterion.to(device)

In [0]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:
        
        optimizer.zero_grad()
        text, text_lengths = batch.text
        predictions = model(text, text_lengths).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:

            text, text_lengths = batch.text
            predictions = model(text, text_lengths).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()

        return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [21]:
best_valid_loss = 1.0

for epoch in range(5):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    print(f'{epoch+1} : train {train_loss}, {train_acc*100:.2f}% | valid {valid_loss}, {valid_acc*100:.2f}%')

1 : train 0.6011528791896851, 65.97% | valid 0.5025404732293758, 74.81%
2 : train 0.500127481333763, 74.46% | valid 0.45282131124391645, 77.63%
3 : train 0.4424834868228768, 78.28% | valid 0.43318716981001315, 79.02%
4 : train 0.4023469900032122, 80.80% | valid 0.4295787651887619, 79.65%
5 : train 0.3711702753387758, 82.69% | valid 0.42979485679982987, 80.02%


In [22]:
model.load_state_dict(torch.load('model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'{test_loss}, {test_acc*100:.2f}%')

0.4360640950718576, 79.21%
